Here's a breakdown of what's happening:

- **Import Libraries:** pandas is imported to manipulate data, and os to work with file names.

- **Excel Files List:** A list xlsx_files contains the names of the Excel files you wish to merge.

- **ExcelWriter Initialization:** pd.ExcelWriter('merged_file.xlsx') is used to create a new Excel file where all the data will be saved.

- **File Loop:** Loop through each .xlsx file in the list.

- **Read Excel File:** Each Excel file is read into a DataFrame using pd.read_excel().

- **Sheet Name:** Extract the file name without the extension to use as the sheet name using os.path.splitext().

- **Write to Excel:** Each DataFrame is written to the Excel writer object with its respective sheet name.

- **Save File:** Once the loop is complete, the Excel writer object saves the file, which will contain each original .xlsx file as a separate sheet.



In [1]:
# Suppress FutureWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [5]:
# Importing Required Libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# Importing User Sensible Data
from sql.postgres_connection import (dbname, password, host, port, database)
from sql.comprasgov_siasg_dw_bps_extraction import (full_sql_query)

# Database Connection Function
def read_data_from_postgres(query):

    df = pd.DataFrame()  # assign a default value to df

    try:
        engine = create_engine(
            f'postgresql://{dbname}:{password}@{host}:{port}/{database}')
        df = pd.read_sql_query(query, engine)

    except Exception as e:
        print("An error occurred:", e)

    return df

# Test Connection
df_structure = read_data_from_postgres(full_sql_query)
# Rename columns of df_structure
df_structure.rename(columns={'it_co_unidade_gestora': 'Cod UResp Compra', 
                             'it_nu_cgc_cpf': 'it_nu_cgc_cpf', 
                             'it_nu_cgc_cpf_orgao': 'it_nu_cgc_cpf_orgao',},
                             inplace=True)

# Make all columns integer
df_structure['Cod UResp Compra'] = df_structure['Cod UResp Compra'].fillna(0).astype(int)
df_structure['it_nu_cgc_cpf'] = df_structure['it_nu_cgc_cpf'].replace('',0).fillna(0)
df_structure['it_nu_cgc_cpf'] = df_structure['it_nu_cgc_cpf'].replace('00000000000000',0).fillna(0)
# Create a new colun 'CNPJ UResp Compra'. If 'CNPJ UResp Compra==0, then use 'it_nu_cgc_cpf_orgao'
df_structure['CNPJ UResp Compra'] = np.where(df_structure['it_nu_cgc_cpf']==0, 
                                             df_structure['it_nu_cgc_cpf_orgao'], 
                                             df_structure['it_nu_cgc_cpf'])

df_structure = df_structure[['Cod UResp Compra', 'CNPJ UResp Compra']]
df_structure_original = df_structure.copy()
df_structure.sample(10)

# df = df_original.copy()

,Cod UResp Compra,CNPJ UResp Compra
30036,686900,0
33690,450645,0
18486,885801,0
4818,158093,09363392000128
884,120439,0
17096,865706,0
4650,153705,0
7967,180005,0
35520,70059,0
19668,925034,27080530000143


In [15]:
import pandas as pd
import os

# Directory where the .xlsx files are located
input_directory = 'data/'

# Directory where the merged .xlsx file will be saved
output_directory = 'output/'

# Make sure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Output file name
output_file_name = 'Ouvidoria 18002.004741.2023-18 - 2018-2023 - Grupo Material 65.xlsx'
output_file_path = os.path.join(output_directory, output_file_name)

# Get the list of .xlsx files you want to merge from the data directory
xlsx_files = [f for f in os.listdir(input_directory) if f.endswith('.xlsx')]

# Initialize a Pandas Excel writer using ExcelWriter class
with pd.ExcelWriter(output_file_path) as writer:

    for xlsx_file in xlsx_files:
        # Read each .xlsx file into a DataFrame
        df = pd.read_excel(os.path.join(input_directory, xlsx_file),
                           header=2,
                           skiprows=1,
                        #    nrows=5,
                           )

        # Join tables
        df_result = df.merge(df_structure, 
                                     how='left', 
                                     left_on='Cod UResp Compra', 
                                     right_on='Cod UResp Compra'
                                     )
        df_result = df_result.drop(columns=['Modalidade Compra', 'Marca Material Compra', 'Métrica', 'Valor Total Homologado'])

        df_result = df_result[['Codigo Material Serviço', 'Descrição Material Servico',
                               'Unidade Fornecimento', 'Dia Resultado Compra', 'Tipo Licitação',
                               'Fabric Material Compra', 'Nome Fornecedor',
                               'CPF/CNPJ Fornecedor', 'Cod UResp Compra', 'Nome UResp Compra',
                               'UF UResp Compra', 'CNPJ UResp Compra', 'Municipio UResp Compra',
                               'Qtde Comprada Item', 'Valor Unitário Homologado']]

        df_result['CPF/CNPJ Fornecedor'] = df_result['CPF/CNPJ Fornecedor'].astype(str)

        # Get the name of the file without the extension
        sheet_name = os.path.splitext(xlsx_file)[0]
        
        # Write each DataFrame to the Excel writer object, specifying the sheet name
        df_result.to_excel(writer, sheet_name=sheet_name, index=False)

# The merged Excel file will be saved in the output/ folder with separate sheets for each original file.